In [155]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [156]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [157]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [158]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [159]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_input_size(self, input_size):
            self.input_size = input_size
        
        def set_weights(self, weights_and_biases):
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.b1 = weights_and_biases[2]
            self.b2 = weights_and_biases[3]
        
        def set_mutation_sigma(self, n):
            self.mutation_sigma = 3 - (2 - 40 / (10 + n))
        
        def predict(self, input):
            return (self.w2.T @ sigmoid(self.w1 @ input.T + self.b1) + self.b2).reshape(-1)
        
        def __add__(self, other):
            w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, self.input_size))
            w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, self.mutation_sigma, 1)

            return w1, w2, b1, b2
    
    def __init__(self, n = 100, random_state = None):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n // 2 * 2
        
        for i in range(self.n):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)
    
    def fit(self, X_train, y_train, epochs = 100):
        self.input_size = X_train.shape[1]

        for key in self.nets.keys():
            self.nets[key].set_input_size(self.input_size)

            self.nets[key].set_weights((
                np.random.uniform(-3, 3, (self.input_size, self.input_size)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, 1))
            )

        for epoch in range(epochs):
            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
                self.nets[key].set_mutation_sigma(epoch)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")

            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
            

    def predict(self, X):
        return self.nets[self.best_net].predict(X)

In [160]:
regressor = ERegressor(n = 100, random_state = 42)
regressor.fit(scaled_X_train, y_train, epochs = 500)

Epoch 0: MAE: 15.84001299310242
Epoch 1: MAE: 7.562097483823801
Epoch 2: MAE: 6.603860556755444
Epoch 3: MAE: 5.459746528421464
Epoch 4: MAE: 5.331289039828886
Epoch 5: MAE: 4.158480010871466
Epoch 9: MAE: 3.950869063840526
Epoch 10: MAE: 3.8419394828204507
Epoch 16: MAE: 3.806088284363756
Epoch 17: MAE: 3.647468044541973
Epoch 24: MAE: 3.379499598077473
Epoch 29: MAE: 3.3152727964280437
Epoch 34: MAE: 3.3027657758942146
Epoch 36: MAE: 3.14099450183403
Epoch 39: MAE: 3.1293323553460026
Epoch 41: MAE: 3.0160824695110464
Epoch 43: MAE: 3.002309281899387
Epoch 48: MAE: 2.957886757391148
Epoch 50: MAE: 2.9150735631307527
Epoch 54: MAE: 2.743381412799993
Epoch 56: MAE: 2.6573199630073057
Epoch 60: MAE: 2.6556392553762254
Epoch 69: MAE: 2.617235306592156
Epoch 76: MAE: 2.608724203737862
Epoch 88: MAE: 2.564461424743371
Epoch 91: MAE: 2.38664613815545
Epoch 130: MAE: 2.380080151118067
Epoch 166: MAE: 2.3597277716339065
Epoch 167: MAE: 2.338362370874452
Epoch 168: MAE: 2.330503194298388
Epoch 

In [161]:
y_pred = regressor.predict(scaled_X_test)

In [162]:
y_pred

array([29.00389089, 32.20654339, 24.37838254, 34.20389475, 15.60094461,
       20.90317433, 14.03378715, 36.57877798, 13.74360083, 23.24980837,
       25.96585388, 24.08280519, 34.73410466, 13.68951605, 28.31213204,
       32.71411793, 16.47937138, 32.26869033, 13.71492158, 22.05728113,
       18.54194395, 19.74567093, 24.45342949, 18.9948164 , 33.78357897,
       21.57228338, 19.73321407, 24.07178973, 28.96486765, 25.32237088,
       17.66744474, 17.94585252, 13.68951478, 22.23965581, 13.69009066,
       33.69120207, 25.12572529, 33.71336932, 26.06699751, 14.74842717])

In [163]:
mean_absolute_error(y_test, y_pred)

2.056062089726409